In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

In [2]:
# === CONFIGURACIÓN DE CREDENCIALES ===
usuario = "AR_POWERBI"
contrasena = "Sim2024plit!"
host = "usasuxdb263p.cotyww.com"             # Ej: "localhost" o "192.168.1.100"
puerto = "1557"              # Puerto típico de Oracle
servicio = "INEVPRO1"     # Ej: "XEPDB1", "ORCL", etc.
# === CREAR EL ENGINE DE SQLAlchemy ===
dsn = f"{host}:{puerto}/{servicio}"
engine = create_engine(f"oracle+oracledb://{usuario}:{contrasena}@{dsn}")

# === TU CONSULTA ===
query = "SELECT EAN,DESCRIPTION, DIVISION, HOUSE FROM VW_PRODUCTO"  # Adaptá según lo que necesites

# === ABRIR UN DBAPI2 CONNECTION Y CARGAR EN DATAFRAME ===
conn = engine.raw_connection()
try:
    simplit_productos = pd.read_sql(query, con=conn)
    print("✅ Carga Exitosa")
finally:
    conn.close()


C:\Users\ArnaldoCarrasco\AppData\Local\Temp\ipykernel_4944\1299574890.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  simplit_productos = pd.read_sql(query, con=conn)


✅ Carga Exitosa


In [3]:
file_path=r"C:\Users\ArnaldoCarrasco\Documents\SO SCRIPT\00.0 Dimensionales\ACTUALIZADOR DE MAESTRO\Buscador EAN.xlsx"
dashboard_producto = pd.read_excel(file_path, sheet_name='Informacion')


In [4]:
# 1) Columnas de interés
cols = ['House', 'SUB-DIVISION']

# 2) Filtrar nulos en SUB-DIVISION, quitar duplicados y reindexar
house_subdivision_unica = (
    dashboard_producto[cols]
    .dropna(subset=['SUB-DIVISION'])    # elimino filas con SUB-DIVISION == NaN
    .drop_duplicates()
    .reset_index(drop=True)
)

# 3) Ver resultado
house_subdivision_unica


,House,SUB-DIVISION
0,ADIDAS,FRAGANCIAS MASIVAS
1,BURBERRY,PRESTIGE
2,CALVIN KLEIN,PRESTIGE
3,COTY,COLOR COSMETICS
4,DAVIDOFF,PRESTIGE
5,GUCCI,PRESTIGE
6,HUGO BOSS,PRESTIGE
7,LACOSTE,PRESTIGE
8,LUXURY,PRESTIGE
9,MARC JACOBS,PRESTIGE


In [5]:
# 1) Convertir a string (los NaN → "nan")
simplit_productos['EAN'] = simplit_productos['EAN'].astype(str)

# 2) Quitar todo lo que no sean dígitos
simplit_productos['EAN'] = simplit_productos['EAN'].str.replace(r'\D+', '', regex=True)

# 3) Reemplazar las cadenas vacías (p.ej. "nan" → "") por "0"
simplit_productos['EAN'] = simplit_productos['EAN'].replace('', '0')

# 4) Convertir a int64
simplit_productos['EAN'] = simplit_productos['EAN'].astype(np.int64)

# 1) Convertir a string (los NaN → "nan")
dashboard_producto['EAN'] = dashboard_producto['EAN'].astype(str)

# 2) Quitar todo lo que no sean dígitos
dashboard_producto['EAN'] = dashboard_producto['EAN'].str.replace(r'\D+', '', regex=True)

# 3) Reemplazar las cadenas vacías por "0"
dashboard_producto['EAN'] = dashboard_producto['EAN'].replace('', '0')

# 4) Convertir a int64
dashboard_producto['EAN'] = dashboard_producto['EAN'].astype(np.int64)



In [6]:
simplit_productos['EAN_CHECK']= simplit_productos['EAN'].isin(dashboard_producto['EAN'])
# 1) Filtrar por EAN_CHECK == False
filtrado = simplit_productos.query("EAN_CHECK == False")

# 2) Renombrar las columnas que vas a traer
filtrado = filtrado.rename(
    columns={
        "DIVISION": "Division",
        "HOUSE":    "House",
        "DESCRIPTION": "Descripcion Producto"
    }
)

# 3) Seleccionar sólo las columnas que querés
columnas_a_retener = ["EAN", "Division", "House", "Descripcion Producto"]
filtrado = filtrado[columnas_a_retener]

# 4) Ahora podés quedarte con el registro más reciente por EAN:
filtrado = filtrado.drop_duplicates(subset=["EAN"])

# Resultado final
filtrado_con_query = filtrado


In [7]:
# Partimos de:
#  - house_subdivision_unica: DataFrame con columnas ['House','SUB-DIVISION']
#  - filtrado_con_query: DataFrame con al menos las columnas ['EAN','Division','House','Descripcion Producto']

# 1) Creamos el diccionario House → SUB-DIVISION (para nuestro "subcategory")
house_to_sub = dict(zip(
    house_subdivision_unica['House'],
    house_subdivision_unica['SUB-DIVISION']
))

# 2) Generamos la columna 'subcategory' mapeando desde 'House'
filtrado_con_query['SUB-DIVISION'] = filtrado_con_query['House'].map(house_to_sub)

# 3) Para los casos donde 'House' esté vacío o NaN, ponemos 'NO HOUSE'
mask_blank_house = (
        filtrado_con_query['House'].isna() |
        (filtrado_con_query['House'].astype(str).str.strip() == '')
)
filtrado_con_query.loc[mask_blank_house, 'SUB-DIVISION'] = 'NO HOUSE'

# 4) (Opcional) Si quieres rellenar cualquier otro NaN que queden tras el mapeo:
filtrado_con_query['SUB-DIVISION'] = filtrado_con_query['SUB-DIVISION'].fillna('SIN SUBCATEGORY')

In [8]:
# Asegurarnos de que subcategory es string (para usar str.lower)
filtrado_con_query['SUB-DIVISION'] = filtrado_con_query['SUB-DIVISION'].astype(str)

# Crear máscara: no nulos y distintos de "sin subcategory" (ignorando mayúsculas)
mask = (
        filtrado_con_query['SUB-DIVISION'].notna() &
        (filtrado_con_query['SUB-DIVISION'].str.strip().str.lower() != 'sin subcategory')
)

# Aplicar y reindexar
filtrado_con_query = (
    filtrado_con_query
    .loc[mask]
    .reset_index(drop=True)
)


In [9]:
filtrado_con_query

,EAN,Division,House,Descripcion Producto,SUB-DIVISION
0,0,CONSUMER BEAUTY,RIMMEL COSMETICS,BRIT MANICURE 443,COLOR COSMETICS
1,7891350047769,CONSUMER BEAUTY,RISQUE,RISQUE- ESM GEL CREM MOCACCINO,COLOR COSMETICS
2,7891350047776,CONSUMER BEAUTY,RISQUE,RISQUE - ESM GEL CREM MACARON AMO,COLOR COSMETICS
3,7891350047783,CONSUMER BEAUTY,RISQUE,RISQUE - ESM GEL CREM NTE ESTRELADA,COLOR COSMETICS
4,7891350047790,CONSUMER BEAUTY,RISQUE,RISQUE - ESM GEL CREM SMTH PESSEGO,COLOR COSMETICS
5,7891350047806,CONSUMER BEAUTY,RISQUE,RISQUE - ESM GEL MET COSTELA D ADAO,COLOR COSMETICS
6,7891350047813,CONSUMER BEAUTY,RISQUE,RISQUE - ESM GEL EF AZ CAPRI,COLOR COSMETICS
7,2049130200001,CONSUMER BEAUTY,MAX FACTOR,MF COD UNI TESTER FY25,COLOR COSMETICS
8,2149852000009,CONSUMER BEAUTY,MAX FACTOR,BANDEJAS 2HFY25 MF 40CM - FLAGSHIP,COLOR COSMETICS
9,7795646622008,CONSUMER BEAUTY,PRET-A-PORTER,COD UNI PAP + RISQUE FY26,FRAGANCIAS MASIVAS


# Futuro Modelado

In [10]:
# # filtrado_con_query: DataFrame de 4 cols ["EAN","Division","House","Descripcion Producto"]
# # dashboard_producto: DataFrame original con muchas columnas, entre ellas las mismas 4
# 
# # 1) Identificar nuevos EANs que no estén ya en dashboard_producto
# mask_nuevos = ~filtrado_con_query['EAN'].isin(dashboard_producto['EAN'])
# nuevos = filtrado_con_query.loc[mask_nuevos].copy()
# 
# # 2) Agregar las columnas faltantes en 'nuevos' con valores nulos (para alinear con dashboard)
# faltantes = [c for c in dashboard_producto.columns if c not in nuevos.columns]
# for col in faltantes:
#     nuevos[col] = pd.NA
# 
# # 3) Reordenar las columnas de 'nuevos' para que coincidan con el orden de dashboard_producto
# nuevos = nuevos[dashboard_producto.columns]
# 
# # 4) Concatenar
# dashboard_producto = pd.concat([dashboard_producto, nuevos], ignore_index=True)


In [11]:
# # Partimos de:
# #  - house_subdivision_unica: DataFrame con columnas ['House', 'SUB-DIVISION']
# #  - dashboard_producto: DataFrame original con muchas columnas, incluyendo 'House' y 'SUB-DIVISION'
# 
# # 1) Creamos un diccionario House → SUB-DIVISION
# house_to_sub = dict(zip(
#     house_subdivision_unica['House'],
#     house_subdivision_unica['SUB-DIVISION']
# ))
# 
# # 2) Rellenamos SUB-DIVISION según el mapping
# dashboard_producto['SUB-DIVISION'] = dashboard_producto['House'].map(house_to_sub)
# 
# # 3) Para filas donde House esté en blanco o NaN, ponemos 'NO HOUSE'
# mask_blank_house = (
#         dashboard_producto['House'].isna() |
#         (dashboard_producto['House'].astype(str).str.strip() == '')
# )
# dashboard_producto.loc[mask_blank_house, 'SUB-DIVISION'] = 'NO HOUSE'
# 
# # Ahora dashboard_producto tiene SUB-DIVISION completado:
# #  - Si House corresponde a una entrada en house_subdivision_unica, usamos ese valor.
# #  - Si House está en blanco/NaN, queda 'NO HOUSE'.
# #  - Si House no aparece en el mapping, SUB-DIVISION quedará NaN (podés rellenarlo si lo necesitás).
